## 1. bot_questions.py

programatically or manually generate a file "questions.csv" with formatted sample questions

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/chateval/archive/master/eval_data/ncm/neural_conv_model_eval_source.txt",
                 delimiter='^',header=None,names=['Question'])

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/chateval/archive/master/eval_data/ncm/neural_conv_model_eval_responses_human_2.txt",
                 delimiter='^',header=None,names=['Answer'])

In [ ]:
pd.concat([df, df2], axis=1, join="inner").to_csv("data/questions.csv", index=False) #do not run

In [ ]:
df = pd.read_csv("data/questions.csv")
df['Vote1'] = 0
df['Vote2'] = 0
df['Vote3'] = 0
df.to_csv("data/votes.csv", index=False)

## 

## 2. bot_test.py

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/questions.csv")

In [ ]:
## loop through first 5 rows,
## no need to do this, use "lambda" instead
for index, row in df[:5].iterrows():
    print(row['Question'], row['Answer'])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BlenderbotTokenizer, BlenderbotForConditionalGeneration
import torch
import IProgress
import requests
import json
def model1(q):
    url = 'http://02ac-69-115-7-84.ngrok.io/webhooks/rest/webhook'
    x = requests.post(url, json = {"message": q})
    a = json.loads(x.text)
    for i in a:
        res = i['text']
        print(res)
        return res
def model2(q):  #https://huggingface.co/microsoft/DialoGPT-medium
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(q + tokenizer.eos_token, return_tensors='pt')
    # append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
def model3(q):
    mname = 'facebook/blenderbot-400M-distill'
    bbmodel = BlenderbotForConditionalGeneration.from_pretrained(mname)
    tokenizer = BlenderbotTokenizer.from_pretrained(mname)
    inputs = tokenizer([q], return_tensors='pt')
    reply_ids = bbmodel.generate(**inputs)
    msg = tokenizer.batch_decode(reply_ids)[0].replace("<s>", "").replace("</s>", "")
    print(msg)
    return msg

In [ ]:
df['Answer_Model1'] = df.apply(lambda row : model1(row['Question']), axis = 1)
df['Answer_Model2'] = df.apply(lambda row : model2(row['Question']), axis = 1)
df['Answer_Model3'] = df.apply(lambda row : model3(row['Question']), axis = 1)

In [ ]:
df['Answer_Model1'] = df.apply(lambda row : model1(row['Question']), axis = 1)

In [ ]:
df['Answer_Model2'] = df.apply(lambda row : model2(row['Question']), axis = 1)

In [ ]:
df['Answer_Model3'] = df.apply(lambda row : model3(row['Question']), axis = 1)

In [ ]:
df

In [ ]:
df.to_csv("data/model_answers.csv", index=False)

## 3. bot_survey.py (Streamlit App)

## 4. bot_grading.py

In [ ]:
import pandas as pd
df = pd.read_csv("data/model_answers_standard.csv")

In [ ]:
df[:3]

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
# model = SentenceTransformer('stsb-roberta-large')
# sentence1 = "I like Python because I can build AI applications"
# sentence2 = "I like Python because I can do data analytics"
# # encode sentences to get their embeddings
# embedding1 = model.encode(sentence1, convert_to_tensor=True)
# embedding2 = model.encode(sentence2, convert_to_tensor=True)
# # compute similarity scores of two embeddings
# cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
# print("Sentence 1:", sentence1)
# print("Sentence 2:", sentence2)
# print("Similarity score:", cosine_scores.item())
def score(v1, v2): #monrovia, Noun: the capital and chief port and largest city of Liberia 
    model = SentenceTransformer('paraphrase-mpnet-base-v2')
    # encode sentences to get their embeddings
    v1 = str(v1)
    v2 = str(v2)
    print("Doing Embeddings of " +v1 +" and " + v2)
    embedding1 = model.encode(v1, convert_to_tensor=True)
    print("first one done")
    embedding2 = model.encode(v2, convert_to_tensor=True)
    print("embeddings done")
    # compute similarity scores of two embeddings
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    print("Sentence 1:", v1)
    print("Sentence 2:", v2)
    print("Similarity score:", cosine_scores.item())
    return cosine_scores.item()

In [ ]:
df['score1'] = df.apply(lambda row : score(row['Answer_Model1'],row['Answer']), axis = 1)
df['score2'] = df.apply(lambda row : score(row['Answer_Model2'],row['Answer']), axis = 1)
df['score3'] = df.apply(lambda row : score(row['Answer_Model3'],row['Answer']), axis = 1)

In [ ]:
df['score1'] = df.apply(lambda row : score(row['Answer_Model1'],row['Answer']), axis = 1)

In [ ]:
df[:10]

In [ ]:
df.to_csv("data/rated_answers.csv", index=False)

### Chungus

In [30]:

from statistics import mean
import pandas as pd
df = pd.read_csv("data/rated_answers.csv")
print("Overall automated scores:")
print(df['score1'].mean())
print(df['score2'].mean())
print(df['score3'].mean())
print("Conversational Subset")
print(df['score1'][0:20].mean())
print(df['score2'][0:20].mean())
print(df['score3'][0:20].mean())
print("General Knowledge Subset")
print(df['score1'][21:40].mean())
print(df['score2'][21:40].mean())
print(df['score3'][21:40].mean())
print("Domain-Specific Subset")
print(df['score1'][41:60].mean())
print(df['score2'][41:60].mean())
print(df['score3'][41:60].mean())
df2 = pd.read_csv("data/votes.csv")
print("Overall human-based scores:")
print(df2['Vote1'].sum())
print(df2['Vote2'].sum())
print(df2['Vote3'].sum())
print("Conversational Subset")
print(df2['Vote1'][0:20].sum())
print(df2['Vote2'][0:20].sum())
print(df2['Vote3'][0:20].sum())
print("General Knowledge Subset")
print(df2['Vote1'][21:40].sum())
print(df2['Vote2'][21:40].sum())
print(df2['Vote3'][21:40].sum())
print("Domain Specific Subset")
print(df2['Vote1'][41:60].sum())
print(df2['Vote2'][41:60].sum())
print(df2['Vote3'][41:60].sum())

Overall automated scores:
0.4639626901944057
0.3869143935646546
0.47386087726612214
Conversational Subset
0.23995939940214156
0.392014766857028
0.35875635631382463
General Knowledge Subset
0.5353623047274979
0.47051736153662205
0.5890043396502733
Domain-Specific Subset
0.6080217756227486
0.3032308214654525
0.43904247548845077
Overall human-based scores:
111
52
71
Conversational Subset
40
24
25
General Knowledge Subset
28
12
23
Domain Specific Subset
35
14
17


In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ['Time', 'Count', 'Number','Session'])
df.to_csv("data/timestamps.csv", index=False)

In [1]:

from statistics import mean
import pandas as pd
df = pd.read_csv("data/timestamps.csv")
df['Session'].unique()

array([ 50353, 389030, 266053,  52133,  90408,  68047, 366892, 805414,
       220232, 134150, 835556, 589673, 952646, 460388, 202229])